In [69]:
from dotenv import load_dotenv

load_dotenv()  # .env 파일을 환경 변수로 로드

True

In [70]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate,MessagesPlaceholder

In [71]:
examples = [
    {
        "movie": "Top Gun",
        "emojis": "🛩️👨‍✈️🔥",
    },
    {
        "movie": "The Godfather",
        "emojis": "👨‍👨‍👦🔫🍝",
    },
    {
        "movie": "The Lion King",
        "emojis": "🦁👑🌍",
    },
    {
        "movie": "The Lord of the Rings",
        "emojis": "🧙‍♂️💍🔥",
    },
]

In [72]:
llm = ChatOpenAI(temperature=0.1)
memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    return_messages=True, # 메세지 형태로 출력
)

In [73]:
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{movie}"),
        ("ai", "{emojis}"),
    ]
)

In [74]:
example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

In [75]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "I'm an emoji master with creativity. Here's a movie expressed with 3 emojis."),
        example_prompt,
        ("human", "{movie}"),
    ]
)

In [80]:
def load_memory(_):
    return memory.load_memory_variables({})["history"]

In [81]:
chain = RunnablePassthrough.assign(history=load_memory) | final_prompt | llm

In [82]:
def invoke_chain(movie):
    result = chain.invoke({"movie": movie})
    memory.save_context(
        {"input": movie},
        {"output": result.content},
    )
    print(result)

In [83]:
invoke_chain("Iron Man")

content='🔧🦸\u200d♂️💥'


In [84]:
invoke_chain("The Dark Knight")

content='🦇🃏🦹\u200d♂️'


In [85]:
invoke_chain("Interstellar")

content='🚀⏳🌌'
